# Which Geyser Gushes First?
http://fivethirtyeight.com/features/which-geyser-gushes-first/

You arrive at the beautiful Three Geysers National Park. You read a placard explaining that the three eponymous geysers — creatively named A, B and C — erupt at intervals of precisely two hours, four hours and six hours, respectively. However, you just got there, so you have no idea how the three eruptions are staggered. Assuming they each started erupting at some independently random point in history, what are the probabilities that A, B and C, respectively, will be the first to erupt after your arrival?

In [1]:
from __future__ import division
import numpy as np
from collections import defaultdict

In [2]:
rng = np.random.RandomState(seed=1337)

In [3]:
def geyser_iteration(intervals):
    """Iteration of the probablistic geyser scenario.
    
    Parameters
    ----------
    intervals : dict, optional
        The intervals that geysers erupt with identifier as dictionary key, default dict(a=2.0, b=4.0, c=6.0).
        
    Returns
    -------
    res : Identifier of the geyser that erupted first.
    """
    starts = {k: rng.uniform(high=interval) for k, interval in intervals.iteritems()}
    max_interval = np.max(intervals.values())
    
    time_arrival = rng.uniform(high=max_interval)
    
    def bests():
        for k, interval in intervals.iteritems():
            eruptions = np.arange(starts[k], max_interval + interval,  interval)
            xs = eruptions - time_arrival
            yield k, xs[xs > 0].min()

    res, _ = reduce(lambda x, y: x if x[1] < y[1] else y, bests())
    return res

In [4]:
def geyser_simulation(iterations=10000, intervals=dict(a=2.0, b=4.0, c=6.0)):
    """Run the simulation to estimate the probability of each geyser's eruption.
    
    Parameters
    ----------
    iterations : int, optional
        Number of iterations to execute for the simulation.
    
    intervals : dict, optional
        The intervals that geyser erupts, default dict(a=2.0, b=4.0, c=6.0).
    
    Returns
    -------
    probabilities : dict
        Probability of each geyser erupting first.
    """
    results = defaultdict(int)
    for _ in range(iterations):
        results[geyser_iteration(intervals)] += 1
    return {a: b / iterations for a, b in results.iteritems()}

In [5]:
geyser_simulation(iterations=10000, intervals=dict(a=2.0, b=4.0, c=6.0))

{'a': 0.6467, 'b': 0.2137, 'c': 0.1396}